<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/AGENTIC-MODULE4-Sensor-Pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
Simple Sensor Pre-Training System
=================================

Trains LSTM Autoencoders for each sensor and saves models with baseline statistics.

Usage:
    python sensor_pretraining.py
"""

import numpy as np
import os
import pickle
from datetime import datetime
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


def build_lstm_autoencoder(window_length: int, latent_dim: int = 32) -> Model:
    """Build LSTM Autoencoder."""
    inputs = Input(shape=(window_length, 1))

    # Encoder
    encoded = LSTM(latent_dim, activation='relu', return_sequences=False)(inputs)

    # Decoder
    decoded = RepeatVector(window_length)(encoded)
    decoded = LSTM(latent_dim, activation='relu', return_sequences=True)(decoded)
    outputs = TimeDistributed(Dense(1, activation='linear'))(decoded)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model


def train_sensor_model(sensor_data, sensor_id, base_path, window_length):
    """Train model for one sensor."""

    print(f"\nTraining sensor {sensor_id}...")
    print(f"Data shape: {sensor_data.shape}")

    # Split data
    n_samples = len(sensor_data)
    n_train = int(0.8 * n_samples)

    X_train = sensor_data[:n_train]
    X_val = sensor_data[n_train:]

    print(f"Train: {len(X_train)}, Val: {len(X_val)}")

    # Create directories
    sensor_dir = os.path.join(base_path, 'sensor', 'model')
    checkpoint_dir = os.path.join(sensor_dir, 'checkpoints')
    os.makedirs(sensor_dir, exist_ok=True)
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Build model
    model = build_lstm_autoencoder(window_length)

    # Setup callbacks
    checkpoint_path = os.path.join(checkpoint_dir, f'sensor_{sensor_id}_best.h5')

    callbacks = [
        # Save best model during training
        ModelCheckpoint(
            filepath=checkpoint_path,
            monitor='val_loss',
            save_best_only=True,
            save_weights_only=False,
            mode='min',
            verbose=1
        ),
        # Early stopping
        EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=1
        ),
        # Reduce learning rate on plateau
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=7,
            min_lr=1e-6,
            verbose=1
        )
    ]

    # Train model
    print("Starting training with checkpointing...")
    history = model.fit(
        X_train, X_train,
        validation_data=(X_val, X_val),
        epochs=100,
        batch_size=32,
        callbacks=callbacks,
        verbose=1
    )

    print(f"Training completed. Best model saved to: {checkpoint_path}")

    # Compute baseline reconstruction errors on validation set
    print("Computing baseline errors...")
    val_predictions = model.predict(X_val, verbose=0)
    baseline_errors = []

    for i in range(len(X_val)):
        error = mean_squared_error(X_val[i].flatten(), val_predictions[i].flatten())
        baseline_errors.append(error)

    baseline_stats = {
        'mean': float(np.mean(baseline_errors)),
        'std': float(np.std(baseline_errors)) + 1e-8,
        'q95': float(np.percentile(baseline_errors, 95)),
        'q99': float(np.percentile(baseline_errors, 99)),
        'baseline_errors': baseline_errors  # Store for drift detection
    }

    # Save final model and metadata
    model_path = os.path.join(sensor_dir, f'sensor_{sensor_id}_model.h5')
    metadata_path = os.path.join(sensor_dir, f'sensor_{sensor_id}_metadata.pkl')

    model.save(model_path)

    metadata = {
        'sensor_id': sensor_id,
        'window_length': window_length,
        'baseline_stats': baseline_stats,
        'trained_at': datetime.now(),
        'epochs_trained': len(history.history['loss']),
        'final_val_loss': float(history.history['val_loss'][-1]),
        'checkpoint_path': checkpoint_path
    }

    with open(metadata_path, 'wb') as f:
        pickle.dump(metadata, f)

    print(f"✅ Sensor {sensor_id} completed")
    print(f"   Final model: {model_path}")
    print(f"   Best checkpoint: {checkpoint_path}")
    print(f"   Baseline error: {baseline_stats['mean']:.6f} ± {baseline_stats['std']:.6f}")

    return baseline_stats


def main():
    """Main training function."""

    # Your exact paths
    data_path = r'/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/multivariate_long_sequences-TRAIN-Daily-DIRECT-VAR.npy'
    base_path = r'/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/'

    print("🚀 Simple Sensor Pre-Training")
    print("=" * 40)

    # Load data
    print("Loading data...")
    data = np.load(data_path)
    print(f"Original data shape: {data.shape}")

    # Remove last 1000 samples (holdout)
    training_data = data[:-1000]
    print(f"Training data shape: {training_data.shape}")

    batch_size, window_length, num_sensors = training_data.shape
    print(f"Will train {num_sensors} sensors")

    # Train each sensor
    print(f"\n🏋️ Training {num_sensors} LSTM Autoencoders...")

    results = {}
    successful = 0

    for sensor_id in range(num_sensors):
        try:
            # Extract data for this sensor: [batch, timestep, 1]
            sensor_data = training_data[:, :, sensor_id:sensor_id+1]  # Keep feature dimension

            baseline_stats = train_sensor_model(sensor_data, sensor_id, base_path, window_length)
            results[sensor_id] = {'success': True, 'baseline_stats': baseline_stats}
            successful += 1

        except Exception as e:
            print(f"❌ Failed training sensor {sensor_id}: {e}")
            results[sensor_id] = {'success': False, 'error': str(e)}

    # Final summary
    print(f"\n📊 TRAINING SUMMARY")
    print("=" * 40)
    print(f"✅ Successful: {successful}/{num_sensors}")
    print(f"💾 Models saved to: {base_path}/sensor/model/")

    if successful > 0:
        print(f"\n🏆 Successfully trained sensors:")
        for sensor_id, result in results.items():
            if result['success']:
                baseline = result['baseline_stats']
                print(f"  Sensor {sensor_id}: baseline error {baseline['mean']:.6f}")

    # Save training summary
    summary_path = os.path.join(base_path, 'sensor', 'model', 'training_summary.pkl')
    with open(summary_path, 'wb') as f:
        pickle.dump({
            'results': results,
            'training_data_shape': training_data.shape,
            'successful_sensors': successful,
            'timestamp': datetime.now()
        }, f)

    print(f"\n💾 Summary saved: {summary_path}")
    print("✅ Pre-training completed!")



if __name__ == "__main__":
    main()

Loading data...
Original data shape: (3627, 50, 12)
Training data shape: (2627, 50, 12)
Training 12 sensors...

Training sensor 0...
Data shape: (2627, 50, 1)
Train: 2101, Val: 526
Epoch 1/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 9s 51ms/step - loss: 0.3428 - val_loss: 0.0599
Epoch 2/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.1577 - val_loss: 0.0472
Epoch 3/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.1252 - val_loss: 0.2405
Epoch 4/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - loss: 0.2178 - val_loss: 0.0569
Epoch 5/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - loss: 0.1385 - val_loss: 0.0522
Epoch 6/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 0.1334 - val_loss: 0.0500
Epoch 7/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - loss: 0.1330 - val_loss: 0.0490
Epoch 8/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - loss: 0.1201 - val_loss: 0.0485
Epoch 9/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.1079 - val_loss: 0.0455
Epoch 10/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 39

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')